# マイクロサービスアーキテクチャ

## 概要

マイクロサービスアーキテクチャ（Microservices Architecture）は、アプリケーションを小さな独立したサービスの集合として構築するアーキテクチャスタイル。

各サービスは独自のプロセスで実行され、軽量なメカニズム（通常はHTTP API）で通信する。

## モノリスとの比較

### モノリシックアーキテクチャ

```
┌────────────────────────────────┐
│                                │
│       モノリシック             │
│      アプリケーション          │
│                                │
│  ┌──────┐ ┌──────┐ ┌──────┐  │
│  │UI    │ │Logic │ │Data  │  │
│  └──────┘ └──────┘ └──────┘  │
│                                │
└────────────────────────────────┘
         単一デプロイメント
```

### マイクロサービスアーキテクチャ

```
┌─────────┐  ┌─────────┐  ┌─────────┐  ┌─────────┐
│User     │  │Order    │  │Payment  │  │Shipping │
│Service  │  │Service  │  │Service  │  │Service  │
│         │  │         │  │         │  │         │
│┌───────┐│  │┌───────┐│  │┌───────┐│  │┌───────┐│
││  DB   ││  ││  DB   ││  ││  DB   ││  ││  DB   ││
│└───────┘│  │└───────┘│  │└───────┘│  │└───────┘│
└────┬────┘  └────┬────┘  └────┬────┘  └────┬────┘
     │            │            │            │
     └────────────┴────────────┴────────────┘
              API Gateway / Service Mesh
```

## マイクロサービスの特徴

### 1. 単一責任の原則

各サービスは特定のビジネス機能に焦点を当てる。

```python
# ユーザーサービス
class UserService:
    def create_user(self, user_data):
        # ユーザー作成のみに責任を持つ
        pass
    
    def get_user(self, user_id):
        pass

# 注文サービス
class OrderService:
    def create_order(self, order_data):
        # 注文作成のみに責任を持つ
        pass
```

### 2. 独立したデプロイメント

各サービスは他のサービスに影響を与えずに独立してデプロイ可能。

### 3. 分散データ管理

各サービスは自身のデータベースを持つ（Database per Service パターン）。

### 4. 技術スタックの多様性

サービスごとに最適な技術を選択可能。

```
User Service: Python + PostgreSQL
Order Service: Java + MySQL
Payment Service: Node.js + MongoDB
```

## サービス間通信

### 同期通信（Synchronous）

**REST API**

```python
# 注文サービスがユーザーサービスを呼び出す
import requests

class OrderService:
    def create_order(self, user_id, items):
        # ユーザー情報を取得
        user_response = requests.get(
            f"http://user-service/api/users/{user_id}"
        )
        user = user_response.json()
        
        # 注文を作成
        order = create_order_logic(user, items)
        return order
```

**gRPC**

```python
# より効率的なバイナリプロトコル
import grpc
import user_pb2
import user_pb2_grpc

channel = grpc.insecure_channel('user-service:50051')
stub = user_pb2_grpc.UserServiceStub(channel)
user = stub.GetUser(user_pb2.UserRequest(id=user_id))
```

### 非同期通信（Asynchronous）

**メッセージキュー**

```python
# RabbitMQ、Kafka、AWS SQSなど
from kafka import KafkaProducer

producer = KafkaProducer(bootstrap_servers='kafka:9092')

# イベントを発行
producer.send('order-created', {
    'order_id': order.id,
    'user_id': user.id,
    'total': order.total
})
```

**イベント駆動**

```python
from kafka import KafkaConsumer

# 別のサービスがイベントを購読
consumer = KafkaConsumer('order-created')

for message in consumer:
    order_data = message.value
    # 在庫を更新、メール送信など
    process_order_created(order_data)
```

## 主要パターン

### 1. API Gateway パターン

クライアントとマイクロサービス間の単一エントリーポイント。

```python
# API Gatewayの例
from fastapi import FastAPI

app = FastAPI()

@app.get("/api/orders/{order_id}")
async def get_order_with_details(order_id):
    # 複数のサービスを呼び出し、集約
    order = await call_order_service(order_id)
    user = await call_user_service(order.user_id)
    payment = await call_payment_service(order_id)
    
    return {
        "order": order,
        "user": user,
        "payment": payment
    }
```

### 2. Service Discovery

サービスの場所を動的に発見。

```python
# Consulを使ったサービス発見
import consul

c = consul.Consul()

# サービスを登録
c.agent.service.register(
    name='order-service',
    address='192.168.1.10',
    port=8080
)

# サービスを発見
services = c.catalog.service('order-service')
```

### 3. Circuit Breaker パターン

障害の伝播を防ぐ。

```python
from pybreaker import CircuitBreaker

breaker = CircuitBreaker(fail_max=5, timeout_duration=60)

@breaker
def call_external_service():
    # 外部サービス呼び出し
    response = requests.get('http://payment-service/api/pay')
    return response.json()
```

### 4. Saga パターン

分散トランザクションの管理。

```python
# Choreography-based Saga
class OrderSaga:
    def create_order(self, order_data):
        # 1. 注文を作成
        order = order_service.create(order_data)
        
        try:
            # 2. 在庫を予約
            inventory_service.reserve(order.items)
            
            # 3. 支払いを処理
            payment_service.charge(order.total)
            
            # 4. 配送を手配
            shipping_service.arrange(order.id)
        except Exception as e:
            # 補償トランザクション
            order_service.cancel(order.id)
            inventory_service.release(order.items)
            raise
```

## メリット

1. **独立したスケーラビリティ**: 負荷の高いサービスのみをスケール
2. **技術的自由度**: サービスごとに最適な技術を選択
3. **障害の分離**: 一つのサービスの障害が全体に波及しない
4. **チームの独立性**: 各チームが独立して開発・デプロイ可能
5. **継続的デリバリー**: 小さな変更を頻繁にリリース可能
6. **理解しやすさ**: 各サービスが小さく、理解しやすい

## デメリット・課題

1. **複雑性の増加**
   - 分散システムの複雑さ
   - サービス間の通信管理

2. **データの一貫性**
   - 分散トランザクションが困難
   - 結果整合性を受け入れる必要

3. **テストの難しさ**
   - 統合テストが複雑
   - エンドツーエンドテストの環境構築が大変

4. **運用コスト**
   - 多数のサービスの監視・ログ管理
   - デプロイメントパイプラインの複雑化

5. **ネットワーク遅延**
   - サービス間通信のオーバーヘッド

6. **データの重複**
   - サービスごとにデータを持つため、重複が発生

## いつマイクロサービスを選ぶべきか

### マイクロサービスが適している場合

- 大規模で複雑なシステム
- 異なる部分が異なる速度で変更される
- 異なる部分が異なるスケーリング要件を持つ
- 複数の独立したチームが開発
- 段階的な技術移行が必要

### モノリスが適している場合

- 小規模なアプリケーション
- 単一チームでの開発
- 初期段階のスタートアップ
- シンプルなビジネスロジック
- 分散システムの経験が少ない

## ツールとテクノロジー

### コンテナ化
- Docker
- Kubernetes

### サービスメッシュ
- Istio
- Linkerd

### API Gateway
- Kong
- AWS API Gateway
- Nginx

### メッセージング
- RabbitMQ
- Apache Kafka
- AWS SQS

### 監視・ログ
- Prometheus + Grafana
- ELK Stack（Elasticsearch, Logstash, Kibana）
- Jaeger（分散トレーシング）